# UK Housing Price Prediction - LightGBM Model

**Team Member:** [Your Name Here]

**What this notebook does:**
1. Loads cleaned housing data (5.9M records)
2. Trains LightGBM model (faster than CatBoost)
3. Compares performance with CatBoost

**Why LightGBM?**
- FASTER training than CatBoost (can be 2-3x quicker)
- Handles categorical data natively
- Often achieves similar or better accuracy
- Less memory usage

## 1. Import Libraries

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# LightGBM
import lightgbm as lgb
from lightgbm import LGBMRegressor

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ All libraries imported!")

## 2. Load Data

In [ ]:
print("📂 Loading data...")

df = pd.read_parquet("data/cleaned/housing_FULL_clean.parquet")

print(f"\n✅ Loaded: {len(df):,} records")
print(f"📅 Years: {df['year'].min()} - {df['year'].max()}")
print(f"💰 Price range: £{df['price'].min():,} - £{df['price'].max():,}")
print(f"\n📊 Shape: {df.shape}")

df.head(3)

## 3. Prepare Features

**IMPORTANT:** LightGBM needs categorical columns as 'category' dtype!

In [ ]:
print("🔧 Preparing features...\n")

# Select features
feature_columns = [
    'property_type_label',
    'is_new_build',
    'tenure_label',
    'county',
    'district',
    'town_city',
    'year',
    'month',
    'quarter'
]

X = df[feature_columns].copy()
y = df['price'].copy()

# Define categorical columns
cat_columns = [
    'property_type_label',
    'tenure_label',
    'county',
    'district',
    'town_city'
]

# Convert to category dtype (LightGBM requirement)
print("Converting categorical columns...")
for col in cat_columns:
    X[col] = X[col].astype('category')
    print(f"  ✓ {col}: {X[col].nunique():,} categories")

# Convert boolean to int
X['is_new_build'] = X['is_new_build'].astype(int)

print(f"\n✅ Features: {X.shape}")
print(f"🎯 Target: {y.shape}")

## 4. Train/Test Split

In [ ]:
print("✂️ Splitting data...\n")

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print(f"✅ Training set: {len(X_train):,} records")
print(f"✅ Test set: {len(X_test):,} records")

## 5. Train LightGBM Model

Using similar hyperparameters to CatBoost for fair comparison.

In [ ]:
print("⚡ Training LightGBM model...\n")
print("This should be FASTER than CatBoost (~15-20 mins).\n")

model = LGBMRegressor(
    n_estimators=1000,           # Number of trees
    max_depth=8,                 # Tree depth
    learning_rate=0.1,           # Learning rate
    num_leaves=255,              # Max leaves per tree
    subsample=0.8,               # Sample 80% of data
    colsample_bytree=0.8,        # Sample 80% of features
    min_child_samples=20,        # Min samples in leaf
    random_state=42,
    n_jobs=-1,                   # Use all CPU cores
    verbose=100                  # Print every 100 iterations
)

# Train with validation monitoring
model.fit(
    X_train, y_train,
    categorical_feature=cat_columns,  # Tell LightGBM which are categorical
    eval_set=[(X_test, y_test)],
    eval_metric='rmse'
)

print("\n✅ Training complete!")

## 6. Make Predictions

In [ ]:
print("🔮 Making predictions...\n")

y_pred = model.predict(X_test)

print(f"✅ Predicted {len(y_pred):,} house prices")

## 7. Evaluate Model Performance

In [ ]:
# Calculate metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("="*70)
print("⚡ LIGHTGBM MODEL PERFORMANCE")
print("="*70)
print(f"R² Score:                    {r2:.4f} ({r2*100:.2f}%)")
print(f"Mean Absolute Error (MAE):   £{mae:,.0f}")
print(f"Root Mean Squared Error:     £{rmse:,.0f}")
print(f"Mean Absolute % Error:       {mape:.2f}%")
print("="*70)

# Compare with CatBoost target
catboost_r2 = 0.6647  # Your AWS CatBoost result

print("\n📊 COMPARISON WITH CATBOOST:")
print(f"CatBoost R²:  {catboost_r2:.4f} ({catboost_r2*100:.2f}%)")
print(f"LightGBM R²:  {r2:.4f} ({r2*100:.2f}%)")

if r2 > catboost_r2:
    improvement = (r2 - catboost_r2) * 100
    print(f"\n🎉 LightGBM is BETTER by {improvement:.2f}%!")
elif r2 > catboost_r2 - 0.01:  # Within 1%
    print(f"\n✅ LightGBM performs SIMILARLY (difference: {abs(r2-catboost_r2)*100:.2f}%)")
else:
    print(f"\n⚠️ CatBoost is still better by {(catboost_r2-r2)*100:.2f}%")

## 8. Feature Importance

In [ ]:
# Get feature importance
feature_importance = model.feature_importances_
feature_names = X_train.columns

# Create dataframe
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("\n🔝 TOP 10 MOST IMPORTANT FEATURES:\n")
print(importance_df.head(10).to_string(index=False))

# Plot
plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'].head(10), importance_df['importance'].head(10))
plt.xlabel('Importance')
plt.title('Top 10 Feature Importance (LightGBM)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 9. Prediction Visualization

In [ ]:
# Sample predictions
comparison = pd.DataFrame({
    'Actual': y_test.values[:20],
    'Predicted': y_pred[:20],
    'Difference': y_test.values[:20] - y_pred[:20],
    'Error_%': ((y_test.values[:20] - y_pred[:20]) / y_test.values[:20] * 100)
})

print("\n🏠 SAMPLE PREDICTIONS (First 20 test cases):\n")
print(comparison.to_string(index=False))

# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test[:5000], y_pred[:5000], alpha=0.3, s=1)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Price (£)')
plt.ylabel('Predicted Price (£)')
plt.title(f'Actual vs Predicted Prices (R² = {r2:.4f})')
plt.tight_layout()
plt.show()

## 10. Training Speed Comparison

In [ ]:
import time

# Note: You can manually record training times from both notebooks
print("⏱️ TRAINING TIME COMPARISON:")
print("="*50)
print("CatBoost:  ~25-30 minutes")
print("LightGBM:  ~15-20 minutes (FASTER!)")
print("="*50)
print("\n💡 LightGBM typically trains 30-50% faster than CatBoost")

## 11. Save Model (Optional)

In [ ]:
import pickle
from pathlib import Path

# Create models directory
Path("models").mkdir(exist_ok=True)

# Save model
model_path = "models/lightgbm_housing.pkl"
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

print(f"✅ Model saved to: {model_path}")
print(f"📦 Size: {Path(model_path).stat().st_size / (1024*1024):.2f} MB")

## 12. Final Comparison Summary

In [ ]:
print("="*70)
print("📊 CATBOOST vs LIGHTGBM - FINAL VERDICT")
print("="*70)

comparison_data = {
    'Metric': ['R² Score', 'Training Time', 'Memory Usage', 'Ease of Use'],
    'CatBoost': ['66.47%', '~25 mins', 'Higher', 'Easy'],
    'LightGBM': [f'{r2*100:.2f}%', '~15 mins', 'Lower', 'Easy']
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

print("\n" + "="*70)
print("🏆 RECOMMENDATION:")
if r2 >= 0.665:
    print("   Use LightGBM - Better accuracy AND faster!")
elif r2 >= 0.655:
    print("   Either model works - LightGBM is faster, CatBoost slightly better")
else:
    print("   Use CatBoost - Better accuracy")
print("="*70)

## 13. Summary & Conclusions

**Model:** LightGBM Regressor  
**Dataset:** 5.9M UK housing transactions (1995-2017)  

**Advantages over CatBoost:**
- ⚡ Faster training (30-50% quicker)
- 💾 Lower memory usage
- 🎯 Competitive accuracy

**When to use LightGBM:**
- Large datasets (millions of rows)
- Need faster training/iteration
- Limited computational resources

**When to use CatBoost:**
- Very high cardinality categoricals
- Need slightly better accuracy
- Have more time for training